In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [2]:
# 
# train = pd.read_csv('data/train.csv', 
#                     parse_dates=['date'], 
#                     index_col='id', 
#                     dtype={
#                         # 'date': np.datetime64, 
#                         'store_nbr': np.short,
#                         'item_nbr': np.int64,
#                         'unit_sales': np.float64
#                     },
#                     converters={'onpromotion': lambda x: 'T' if x == 'True' else ('F' if x == 'False' else 'U')}
#                    )
#
# train.merge(items).to_parquet('data/train_items.parquet')
# train['class'] = train['class'].astype('str')
# train['item_nbr'] = train['item_nbr'].astype('str')
# train.to_parquet('data/train.parquet')

In [14]:
# scikit-based

train_all = pd.read_parquet('data/train_items.parquet')
train_all.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,2013-01-01,25,103665,7.0,U,BREAD/BAKERY,2712,1
1,2013-01-02,1,103665,2.0,U,BREAD/BAKERY,2712,1
2,2013-01-02,2,103665,5.0,U,BREAD/BAKERY,2712,1
3,2013-01-02,3,103665,6.0,U,BREAD/BAKERY,2712,1
4,2013-01-02,4,103665,2.0,U,BREAD/BAKERY,2712,1


In [15]:
train_all.shape

(125497040, 8)

In [3]:
# train['Weekday'] = train['date'].dt.dayofweek_str
train.head()['date'].dt.dayofweek

0    1
1    2
2    2
3    2
4    2
Name: date, dtype: int64

In [9]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils import indexable

from datetime import datetime, timedelta

# TODO
class TemporalSplit(TimeSeriesSplit):
    def __init__(self, step=timedelta(days=1), n_splits=5, *, max_train_size=None, test_size=None, gap=0):
        super().__init__(n_splits)
        self.step = step
        self.max_train_size = max_train_size
        self.test_size = test_size
        self.gap = gap

    def _create_date_ranges(self, start, end, step):
        start_ = start
        while start_ < end:
            end_ = start_ + step
            yield start_
            start_ = end_
            
    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        X, y, groups = indexable(X, y, groups)
        
        date_range = list(self._create_date_ranges(X.index.min(), X.index.max(), self.step))
        n_samples =  len(date_range)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        gap = self.gap
        test_size = (
            self.test_size if self.test_size is not None else n_samples // n_folds
        )

        # Make sure we have enough samples for the given split parameters
        if n_folds > n_samples:
            raise ValueError(
                f"Cannot have number of folds={n_folds} greater"
                f" than the number of samples={n_samples}."
            )
        if n_samples - gap - (test_size * n_splits) <= 0:
            raise ValueError(
                f"Too many splits={n_splits} for number of samples"
                f"={n_samples} with test_size={test_size} and gap={gap}."
            )

        # indices = np.arange(n_samples)
        test_starts = range(n_samples - n_splits * test_size, n_samples, test_size)

        for test_start in test_starts:
            train_end = test_start - gap
            if self.max_train_size and self.max_train_size < train_end:
                yield (
                    # TODO: unit test
                    # TODO: not sure why np.where returns a tuple.
                    np.where(np.logical_and(X.index >= date_range[train_end - self.max_train_size], X.index <= date_range[train_end - 1]))[0],
                    np.where(np.logical_and(X.index >= date_range[test_start], X.index <= date_range[test_start + test_size - 1]))[0]
                    # indices[train_end - self.max_train_size : train_end],
                    # indices[test_start : test_start + test_size],
                )
            else:
                yield (
                    np.where(X.index < date_range[train_end])[0],
                    np.where(np.logical_and(X.index >= date_range[test_start], X.index <= date_range[test_start + test_size - 1]))[0]
                    # indices[:train_end],
                    # indices[test_start : test_start + test_size],
                )

# cv = list(TemporalSplit(n_splits=3, test_size=100).split(train_idx))
# for s in cv:
#     print(s[0])
#     print(s[0][0])
#     print(f'Train: {train_idx.iloc[s[0]].index.min()} - {train_idx.iloc[s[0]].index.max()}')
#     print(f'Test:  {train_idx.iloc[s[1]].index.min()} - {train_idx.iloc[s[1]].index.max()}')
#     print()
    
# cv = list(TimeSeriesSplit(n_splits=3, test_size=100).split(train_idx))
# for s in cv:
#     print(s[0])
#     print(s[0][0])
#     print(f'Train: {train_idx.iloc[s[0]].index.min()} - {train_idx.iloc[s[0]].index.max()}')
#     print(f'Test:  {train_idx.iloc[s[1]].index.min()} - {train_idx.iloc[s[1]].index.max()}')
#     print()

In [101]:
train = train_all.sample(int(10e6))
train = train.set_index('date', drop=False).sort_index()
train.loc[train['unit_sales'] < 0, 'unit_sales'] = 0
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
date,,,,,,,,
2013-01-01,2013-01-01,25,638327,2.0,U,GROCERY I,1084,0
2013-01-01,2013-01-01,25,464385,7.0,U,BREAD/BAKERY,2718,1
2013-01-01,2013-01-01,25,360314,5.0,U,BREAD/BAKERY,2702,1
2013-01-01,2013-01-01,25,557256,10.0,U,EGGS,2502,1
2013-01-01,2013-01-01,25,358515,2.0,U,GROCERY I,1024,0


In [130]:
# train = train_all.copy()
# train = train.set_index('date', drop=False).sort_index()
# train.loc[train['unit_sales'] < 0, 'unit_sales'] = 0

train.shape

(10000000, 8)

In [134]:
from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
import time
import cbm
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
import calendar

class DateEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, feature_name, component = 'month' ):
        self.feature_name = feature_name
        
        if component == 'day':
            self.categories = calendar.day_abbr
            self.column_to_ordinal = lambda col: col.dayofweek.values
        elif component == 'month':
            self.categories = calendar.month_abbr
            self.column_to_ordinal = lambda col: col.month.values
        else:
            raise ValueError('component must be either day or month')
        
        self.component = component
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return self.column_to_ordinal(X.iloc[:,0].dt)[:,np.newaxis]

# Talk to Ilya about this use-case
cats = make_column_transformer(
    # TODO: pass pipeline to CBM model + inspect pipeline to correlate for plotting
    
    # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
    (OrdinalEncoder(dtype='int', handle_unknown='use_encoded_value', unknown_value=-1), # +1 in CBM code
     ['store_nbr', 'item_nbr', 'onpromotion', 'family', 'class', 'perishable']),
       
    (DateEncoder('month', 'month'), ['date']),
    (DateEncoder('day', 'day'),     ['date'])
    # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html
    # (KBinsDiscretizer(n_bins=10, encode='ordinal', dtype='int'),     [''])
)

cbm = cbm.CBM(learning_rate_step_size=1/500, min_iterations_early_stopping=10)

pipeline = make_pipeline(
        cats,
        cbm
    )

# model.fit(x_train, train['unit_sales'])
# pipeline.fit(train.head(10), train['unit_sales'].head(10))
# pipeline.fit(train.head(100000), train['unit_sales'].head(100000))


# pipeline.fit(train, train['unit_sales'])
# 

# from sklearn.model_selection import cross_val_score

start = time.time()
scores = cross_val_score(pipeline, train, train['unit_sales'], 
                         scoring=make_scorer(mean_squared_log_error, squared=False), 
                         cv=TemporalSplit(n_splits=3, test_size=90),
                         n_jobs=-1
                        )

print(f'cross-val { time.time() - start}')
print(scores)

cross-val 53.315948724746704
[0.76249221 0.74114169 0.73643648]


In [31]:
(train['unit_sales'] < 0).sum()

cross-val 53.586721658706665
[0.76333752 0.74206952 0.73735652]

0

In [9]:
pipeline.fit(train, train['unit_sales'])
1

1

In [42]:
train = train_all.copy()
train.loc[train['unit_sales'] < 0, 'unit_sales'] = 0

pipeline_feat = make_pipeline(cats)
pipeline_feat.fit(train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(dtype='int',
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['store_nbr', 'item_nbr',
                                                   'onpromotion', 'family',
                                                   'class', 'perishable']),
                                                 ('dateencoder-1',
                                                  DateEncoder(feature_name='month'),
                                                  ['date']),
                                                 ('dateencoder-2',
                                                  DateEncoder(component='day',
                                            

In [40]:
items

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1
...,...,...,...,...
4095,2132318,GROCERY I,1002,0
4096,2132945,GROCERY I,1026,0
4097,2132957,GROCERY I,1068,0
4098,2134058,BEVERAGES,1124,0


In [123]:
items = pd.read_csv('data/items.csv')

test = (pd.read_csv('data/test.csv', 
                    parse_dates=['date'], 
                    # index_col='id', 
                    dtype={
                        'id':        np.int64,
                        'store_nbr': np.short,
                        'item_nbr': np.int64,
                        'unit_sales': np.float64
                    },
                    converters={'onpromotion': lambda x: 'T' if x == 'True' else ('F' if x == 'False' else 'U')}
                   )
    .merge(items, how='left'))

# test['class']    = test['class'].astype('str')
# test['item_nbr'] = test['item_nbr'].astype('str')

print(test.shape)
print(test.dtypes)
test.head()

(3370464, 8)
id                      int64
date           datetime64[ns]
store_nbr               int16
item_nbr                int64
onpromotion            object
family                 object
class                   int64
perishable              int64
dtype: object


,id,date,store_nbr,item_nbr,onpromotion,family,class,perishable
0,125497040,2017-08-16,1,96995,F,GROCERY I,1093,0
1,125497041,2017-08-16,1,99197,F,GROCERY I,1067,0
2,125497042,2017-08-16,1,103501,F,CLEANING,3008,0
3,125497043,2017-08-16,1,103520,F,GROCERY I,1028,0
4,125497044,2017-08-16,1,103665,F,BREAD/BAKERY,2712,1


In [45]:
# pipeline_feat.transform(test)
test.isna().sum(axis=1).sum()

0

In [124]:
pipeline_feat = make_pipeline(cats)
features = pipeline_feat.transform(test)
features

array([[ 0],
       [ 1],
       [ 2],
       ...,
       [-1],
       [-1],
       [-1]])

In [135]:
# training

train_all_all = train_all.copy()
train_all_all.loc[train_all_all['unit_sales'] < 0, 'unit_sales'] = 0

pipeline_all = make_pipeline(cats, cbm)
pipeline_all.fit(train_all_all, train_all_all['unit_sales'])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(dtype='int',
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['store_nbr', 'item_nbr',
                                                   'onpromotion', 'family',
                                                   'class', 'perishable']),
                                                 ('dateencoder-1',
                                                  DateEncoder(feature_name='month'),
                                                  ['date']),
                                                 ('dateencoder-2',
                                                  DateEncoder(component='day',
                                            

In [136]:
y_pred_test = pipeline_all.predict(test)

pd.Series(y_pred_test.flatten()).describe()

count    3.370464e+06
mean     7.313644e+00
std      1.613676e+01
min      3.352459e-01
25%      2.642382e+00
50%      4.420249e+00
75%      7.781694e+00
max      2.224308e+03
dtype: float64

In [137]:
pd.Series(y_pred_test.flatten()).value_counts()

7.535582    21
7.772725    21
5.474212    21
5.866997    21
7.266502    21
            ..
2.044499     1
9.149783     1
6.819987     1
2.328584     1
8.180235     1
Length: 1539115, dtype: int64

In [138]:
submission = test.copy()
submission.loc[:, 'unit_sales'] = y_pred_test.flatten()
submission[['id', 'unit_sales']].to_csv('submission.csv', index=False)

In [128]:
!head submission.csv

id,unit_sales
125497040,2.5024013377093564
125497041,4.222599293880319
125497042,4.873269254461008
125497043,4.187582054240339
125497044,4.932258731082357
125497045,13.245315426347876
125497046,21.230761691507237
125497047,14.97772186798399
125497048,6.772408878599083


In [139]:
!kaggle competitions submit -c favorita-grocery-sales-forecasting -f submission.csv -m sklearn1

100%|██████████████████████████████████████| 91.2M/91.2M [00:17<00:00, 5.61MB/s]
Successfully submitted to Corporación Favorita Grocery Sales Forecasting

In [32]:
pipeline.predict(train.head())

array([[9.46390226],
       [9.46412469],
       [9.48171461],
       [9.48268984],
       [9.46475154]])

In [18]:
train.shape

(125497040, 8)

In [54]:
from collections import defaultdict

item_store_map = defaultdict(int)

# have the first item as back-off
item_store_map.update({(row['item_nbr'], row['store_nbr']): idx + 1 for idx, row in train[['item_nbr','store_nbr']].value_counts(ascending=True).reset_index(name='count').query('count > 5').iterrows()})

len(item_store_map)

172197

In [55]:
train['item_store'] = train[['item_nbr','store_nbr']].apply(lambda x: item_store_map[tuple(x)], axis=1)

In [3]:
train = pd.read_parquet('data/train_items.parquet')

class_map = {x: i for i, x in enumerate(np.sort(train['class'].unique()))}
item_nbr_map = {x: i for i, x in enumerate(np.sort(train['item_nbr'].unique()))}

train['class']    = train['class'].map(class_map)
train['item_nbr'] = train['item_nbr'].map(item_nbr_map)

train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,2013-01-01,25,4,7.0,U,BREAD/BAKERY,184,1
1,2013-01-02,1,4,2.0,U,BREAD/BAKERY,184,1
2,2013-01-02,2,4,5.0,U,BREAD/BAKERY,184,1
3,2013-01-02,3,4,6.0,U,BREAD/BAKERY,184,1
4,2013-01-02,4,4,2.0,U,BREAD/BAKERY,184,1


In [4]:

# # train['unit_sales'] = train['unit_sales'].astype(np.int32)
# train[['unit_sales']].to_parquet('data/train_unit_sales.parquet')

# x_train = pd.read_parquet('data/train_items_featurized.parquet')
# x_train_unit_sales = pd.read_parquet('data/train_unit_sales.parquet')

In [5]:
(((train['unit_sales'] * 10) % 10  ) > 0).sum()

8191095

In [6]:
8191095 / len(train)

0.06526922866069192

In [7]:
import cbm
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

def featurize(df):
   return pd.DataFrame({
       'store_nbr'    : df['store_nbr'],
       'item_nbr'     : df['item_nbr'],
       'onpromotion'  : df['onpromotion'],
       'family'       : df['family'],
       'class'        : df['class'],
       'perishable'   : df['perishable'],
       'date'         : df['date'],
    })

start = time.time()

x_train = featurize(train)

print(f'featurize { time.time() - start}')

# enable_bin_count=True) # 
model = cbm.CBM(learning_rate_step_size=1/64000, min_iterations_early_stopping=2)
model.fit(x_train, train['unit_sales'])

print(f'train      {time.time() - start}')
print(f'iterations {model.iterations}')

y_pred_train = model.predict(x_train).flatten()

rmsle = mean_squared_error(train['unit_sales'], y_pred_train, squared=False)
print(rmsle)

# 29sec - for store/item
# 76sec - for store/onprom/family/class/perishable - 612k
# 589k w/  1/2000 learning rate
# 612k w/   1/200
# 132  w/  1/4000
# 37   w/  1/8000
# 25   w/ 1/16000 
# 23   w  1/32000 it>=15
# 23   w  1/32000 it>=5
# 23   w  1/64000 it>=2
# 23.60 w 1/64000 it>=2 + item_nbr
# 23.60 w 1/64000 it>=2 + item_nbr + date

featurize 1.988840103149414
train      59.32568883895874
iterations 3
23.605482539271893


In [9]:
x_train['class'].max()

333

In [10]:
x_train['item_nbr'].max()

4035

In [8]:
# model.plot_importance(figsize=(20, 20))

In [45]:
items = pd.read_csv('data/items.csv')

test = pd.read_csv('data/test.csv',
            parse_dates=['date'], 
            index_col='id', 
            dtype={
                # 'date': np.datetime64, 
                'store_nbr': np.short,
                'item_nbr': np.int64,
                'unit_sales': np.float64
            },
).merge(items)

test['onpromotion'] = test['onpromotion'].map({True: 'T', False: 'F'})
test['class']       = test['class'].map(class_map)
test['item_nbr']    = test['item_nbr'].map(item_nbr_map)

test.head()

/home/marcozo/miniconda3/envs/cbm/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date,store_nbr,item_nbr,onpromotion,family,class,perishable
0,2017-08-16,1,0.0,F,GROCERY I,63.0,0
1,2017-08-16,2,0.0,F,GROCERY I,63.0,0
2,2017-08-16,3,0.0,F,GROCERY I,63.0,0
3,2017-08-16,4,0.0,F,GROCERY I,63.0,0
4,2017-08-16,5,0.0,F,GROCERY I,63.0,0


In [53]:
# TODO: handle NA by multiplying by 1
test['item_nbr'] = test['item_nbr'].fillna(0).astype(int)
test['class']    = test['class']   .fillna(0).astype(int)
test[test.isna().any(axis=1)]

,date,store_nbr,item_nbr,onpromotion,family,class,perishable


In [12]:
# class_cats = train_raw['class'].astype('category').cat.categories.tolist()

# test['class'] = test['class'].astype(pd.CategoricalDtype(categories=class_cats, ordered=True)).cat.codes
# test.head()

In [54]:
x_test = featurize(test)
x_test.head()

,store_nbr,item_nbr,onpromotion,family,class,perishable,date
0,1,0,F,GROCERY I,63,0,2017-08-16
1,2,0,F,GROCERY I,63,0,2017-08-16
2,3,0,F,GROCERY I,63,0,2017-08-16
3,4,0,F,GROCERY I,63,0,2017-08-16
4,5,0,F,GROCERY I,63,0,2017-08-16


In [55]:
y_pred_test = model.predict(x_test, explain=True) #.flatten()
y_pred_test

array([[8.67074847, 1.00170424, 1.00180464, ..., 1.00164436, 1.00164009,
        1.00161252],
       [8.67054548, 1.00168079, 1.00180464, ..., 1.00164436, 1.00164009,
        1.00161252],
       [8.67063081, 1.00169065, 1.00180464, ..., 1.00164436, 1.00164009,
        1.00161252],
       ...,
       [8.67042574, 1.00148041, 1.00180464, ..., 1.00164436, 1.0016697 ,
        1.00161252],
       [8.67126881, 1.00157779, 1.00180464, ..., 1.00164436, 1.0016697 ,
        1.00161252],
       [8.67073749, 1.00151642, 1.00180464, ..., 1.00164436, 1.0016697 ,
        1.00161252]])

In [57]:
pd.Series(y_pred_test[:,0].flatten()).value_counts()

8.666806    21
8.666954    21
8.666864    21
8.667392    21
8.666751    21
            ..
8.656977     1
8.659257     1
8.656693     1
8.656806     1
8.641494     1
Length: 1538737, dtype: int64

In [58]:
y_pred_test = model.predict(x_test)

In [64]:
test[['unit_sales']].index.rename('id', inplace=True)
test['unit_sales'] = y_pred_test
test[['unit_sales']].to_csv('submission.csv', index=True)

In [65]:
test['unit_sales'] = y_pred_test
test[['unit_sales']].to_csv('submission.csv', index=True)

In [66]:
!head submission.csv

id,unit_sales
0,8.670748466911578
1,8.670545482030695
2,8.670630814956288
3,8.67052676822758
4,8.66955171527891
5,8.670202952780572
6,8.670841233962294
7,8.670437318412505
8,8.669850978368242


In [20]:
pd.Series(y_pred_train).value_counts()

8.665899    28
8.649263    26
8.649260    26
8.649381    24
8.649342    22
            ..
8.652211     1
8.659009     1
8.664719     1
8.652154     1
8.634132     1
Length: 20537444, dtype: int64

In [67]:
!kaggle competitions submit -c favorita-grocery-sales-forecasting -f submission.csv -m v1

100%|██████████████████████████████████████| 81.9M/81.9M [00:21<00:00, 3.94MB/s]
Successfully submitted to Corporación Favorita Grocery Sales Forecasting